In [1]:
import pandas as pd
import requests
import re
import json
from datetime import datetime, timedelta, date
import sqlalchemy

# creating a function for weather data (dynamic sql table) that takes a list of city names as an argument
def get_wea(cities:list):

    API_key = "XXX" # API key generated by OpenWeatherMap for 5 day weather forecast

    # creating the weather dictionary with needed columns for the SQL table
    wea_dict = {"city_id": [],
                "forecast_time": [],
                "weather_outlook": [],
                "temperature": [],
                "feels_like_temperature": [],
                "wind_speed": [],
                "risk_of_rain": []
                }

    for row in cities.itertuples():
        url = f"http://api.openweathermap.org/data/2.5/forecast?q={row.city}&appid={API_key}&units=metric"
        response = requests.get(url)
        wea_resp = response.json()

        for wea in wea_resp["list"]:
            wea_dict["city_id"].append(row.city_id)
            wea_dict["forecast_time"].append(wea["dt_txt"])
            wea_dict["weather_outlook"].append(wea["weather"][0]["main"])
            wea_dict["temperature"].append(wea["main"]["temp"])
            wea_dict["feels_like_temperature"].append(wea["main"]["feels_like"])
            wea_dict["wind_speed"].append(wea["wind"]["speed"])
            wea_dict["risk_of_rain"].append(wea["pop"])

    return pd.DataFrame(wea_dict)
    
# creating a function for flights data (dynamic sql table)
def tomorrows_flight_arrivals(icao_list):

    today = datetime.now().date()
    tomorrow = today + timedelta(days=1)

    list_for_arrivals_df = []

    for icao in icao_list:
        times = [["00:00","11:59"],["12:00","23:59"]]

        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            querystring = {"direction":"Arrival","withCancelled":"false"}

            headers = {
                "X-RapidAPI-Key": "0b7cffd425mshbf932b1b5f7e633p187a96jsna5aedb4ce276",
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }

            response = requests.request("GET", url, headers=headers, params=querystring)
            flights_resp = response.json()

            arrivals_df = pd.json_normalize(flights_resp["arrivals"])[["number", "airline.name", "movement.scheduledTime.local", "movement.terminal", "movement.airport.name", "movement.airport.icao"]]
            arrivals_df = arrivals_df.rename(columns={"number": "flight_number", "airline.name": "airline", "movement.scheduledTime.local": "arrival_time", "movement.terminal": "arrival_terminal", "movement.airport.name": "departure_city", "movement.airport.icao": "departure_airport_icao"})
            arrivals_df['arrival_airport_icao'] = icao
            arrivals_df['data_retrieved_on'] = datetime.now().date()
            arrivals_df = arrivals_df[["flight_number", "departure_airport_icao", "arrival_airport_icao", "arrival_time"]]
            arrivals_df["arrival_time"] = arrivals_df["arrival_time"].str[:-6]
            list_for_arrivals_df.append(arrivals_df)

    return pd.concat(list_for_arrivals_df, ignore_index=True)

# creating the lambda function on AWS Lambda
def lambda_handler(event, context):
    schema="gans"
    host="wbs-project3-db.cunuvsto5hvy.us-east-1.rds.amazonaws.com"
    user="admin"
    password="XXX"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
  
    city_df = pd.read_sql('cities',con=con)
  
    df_weathers = get_wea(city_df)
    df_weathers.to_sql('weathers',if_exists='append',con=con,index=False)
    
    df_flights = tomorrows_flight_arrivals(["EDDB", "EDDF", "EDDH", "EDDK", "EDDM"])
    df_flights.to_sql('flights',if_exists='append',con=con,index=False)
  
    return {
    'statusCode': 200,
    'body': json.dumps('Hello from Lambda!')}